In [1287]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [1288]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 80

In [1289]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [1290]:
def select_season (df, season):
    return df[df['season']==season]

In [1291]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [1292]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [1293]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [1294]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]
     
   

    return X_train, Y_train, X_test, Y_test

In [1864]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    lis_2=[]
    lis_4=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis_2=lis_2+list(method.predict(X_test))
        lis_4=lis_4+list(Y_test)
        lis.append(float(method.score(X_test, Y_test)))
        #print(X_test)
        #print(method.predict_proba(X_test))
        #print(Y_test)

    tmp[method_name]=lis 
    
    tmp_2=BIG.loc[list(test_index)].iloc[:,[1,2,3,4,8]]
    
    
    
    tmp_2['prediction']=lis_2
    tmp_2['real']=lis_4
    
    return tmp ,tmp_2

In [1865]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


In [2031]:
r=allchampionships[9].df#[abs(allchampionships[9].df.iloc[:,[21,22,38,39]])>0]
#allchampionships[9].df.iloc[:,[21,22,38,39]]

In [2038]:
weight={}
column_to_use_as_prediction=[60,61,62,63]
#weight='balanced'
#weight={1:0.3403,0:0.28 }

In [2039]:
svm_linear = svm.SVC(kernel='linear',probability=True, class_weight=weight)

In [2040]:
#allchampionships[9].df

In [2044]:
a,b=Predict_whole_season_by_one (r, season_to_be_predicted, column_to_use_as_prediction, 'Goal/No_Goal', svm_linear, 'svm')

In [2045]:
a

,svm
0,0.6
1,0.5
2,0.5
3,0.6
4,0.6
5,0.4
6,0.3
7,0.0
8,0.5
9,0.4


In [2042]:
a.mean()

svm    0.543333
dtype: float64

In [1974]:
b['accuracy']=(b['prediction']==b['real'])*1

In [1975]:
b['real'].value_counts()

0    163
1    137
Name: real, dtype: int64

In [1976]:
b['prediction'].value_counts()

1    163
0    137
Name: prediction, dtype: int64

In [1952]:
b['accuracy'].value_counts()

0    163
1    137
Name: accuracy, dtype: int64

In [1831]:
b[b.H_ROUND==5]

,season,Date,HomeTeam,AwayTeam,H_ROUND,prediction,real,accuracy
2677,2015/2016,2015-09-22 00:00:00,Udinese,Milan,5.0,0,1,0
2678,2015/2016,2015-09-23 00:00:00,Sampdoria,Roma,5.0,0,1,0
2679,2015/2016,2015-09-23 00:00:00,Carpi,Napoli,5.0,0,0,1
2680,2015/2016,2015-09-23 00:00:00,Fiorentina,Bologna,5.0,0,0,1
2681,2015/2016,2015-09-23 00:00:00,Inter,Hellas Verona,5.0,0,0,1
2682,2015/2016,2015-09-23 00:00:00,Chievo Verona,Torino,5.0,0,0,1
2683,2015/2016,2015-09-23 00:00:00,Lazio,Genoa,5.0,0,0,1
2684,2015/2016,2015-09-23 00:00:00,Palermo,Sassuolo,5.0,0,0,1
2685,2015/2016,2015-09-23 00:00:00,Juventus,Frosinone,5.0,0,0,1
2686,2015/2016,2015-09-24 00:00:00,Empoli,Atalanta,5.0,0,0,1


In [1718]:
b[b['accuracy']==0]

,season,Date,HomeTeam,AwayTeam,H_ROUND,prediction,real,accuracy
1262,2015/2016,2015-08-12 00:00:00,BSC Young Boys,Lugano,5.0,1,0,0
1263,2015/2016,2015-08-12 00:00:00,FC Luzern,FC St. Gallen,5.0,1,0,0
1270,2015/2016,2015-08-23 00:00:00,FC St. Gallen,Grasshopper Club Zürich,6.0,1,0,0
1272,2015/2016,2015-08-29 00:00:00,Lugano,FC Luzern,7.0,1,0,0
1273,2015/2016,2015-08-29 00:00:00,FC Vaduz,FC St. Gallen,7.0,1,0,0
1276,2015/2016,2015-08-30 00:00:00,FC Thun,FC Sion,7.0,1,0,0
1277,2015/2016,2015-09-12 00:00:00,BSC Young Boys,FC Vaduz,8.0,1,0,0
1280,2015/2016,2015-09-13 00:00:00,FC Sion,Lugano,8.0,1,0,0
1284,2015/2016,2015-09-23 00:00:00,Grasshopper Club Zürich,FC Sion,9.0,1,0,0
1289,2015/2016,2015-09-27 00:00:00,FC Sion,FC Vaduz,10.0,1,0,0


In [1698]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    lis_2=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [1310]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        lis2.appen
        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [1311]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

In [2048]:
def pred_all_meth (df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction):
    
    lis=[]
    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'
    
    svm_linear = svm.SVC(kernel='linear', class_weight={'H':66,'D':35,'A':39})
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear_3')))    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'
    
    svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf_3')))

    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict = 'Goal/No_Goal'  
               
    #dtc = DecisionTreeClassifier()
    #lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_1')))
    #lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_3')))

    
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'  
               
    #bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
    #lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_1')))
    #lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_3')))          
               
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'           
               
    #gradb = GradientBoostingClassifier(max_depth=10, n_estimators=20)
    #lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_1')))
    #lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_3')))         
   
                    
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'  
                    
    Ardf = AdaBoostClassifier(RandomForestClassifier(max_depth=10, n_estimators=20))
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf_3')))
    
    
    
    return lis

In [2049]:
def Put_Togheter (lis):
    tmp=pd.DataFrame()
    for i in range(len(lis)):
        tmp=tmp.join(lis[i].df, how='outer')
        
    return tmp

# Loading data

In [2050]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [2051]:
percentages=pd.read_csv('../Data/1x2_all_leagues')
percentages

,Unnamed: 0,Switzerland Super League,Poland Ekstraklasa,Scotland Premier League,France Ligue 1,Germany 1. Bundesliga,England Premier League,Belgium Jupiler League,Portugal Liga ZON Sagres,Netherlands Eredivisie,Italy Serie A,Spain LIGA BBVA
0,0,0.452830,0.454812,0.429167,0.444583,0.454885,0.460899,0.467764,0.442890,0.486371,0.466056,0.493539
1,1,0.243486,0.275235,0.240972,0.288333,0.243994,0.258735,0.239369,0.254662,0.230893,0.267805,0.227178
2,2,0.303684,0.269953,0.329861,0.267083,0.301121,0.280366,0.292867,0.302448,0.282737,0.266139,0.279283


In [2052]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [2053]:
n=4
m=4

In [2054]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [2055]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


# Parameter

In [2056]:
season_to_be_predicted='2015/2016'
column_to_predict='Goal/No_Goal'
column_to_use_as_prediction = [56,57]


# RUN with all method only ONE LEAGUE

In [2047]:
a=pred_all_meth(allchampionships[1].df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

## Put together

In [ ]:
b=Put_Togheter(a)

In [ ]:
b

In [ ]:
b.mean()

## Get Means and Best Methods

In [773]:
for i in range(len(b.mean()[b.mean()==max(b.mean())])):
    print ('Best method are', b.mean()[b.mean()==max(b.mean())].index[i], 'with score', b.mean()[b.mean()==max(b.mean())][i])


('Best method are', 'svm_linear_1', 'with score', 0.37375073486184601)
('Best method are', 'svm_linear_2', 'with score', 0.37375073486184601)
('Best method are', 'svm_rbf_1', 'with score', 0.37375073486184601)
('Best method are', 'svm_rbf_2', 'with score', 0.37375073486184601)


# Run all methods ALL LEAGUES 

In [1284]:
def FINAL (df,season_to_be_predicted,column_to_predict,column_to_use_as_prediction):
    lis=[]
    tmp=pd.DataFrame(columns=[u'svm_linear_1', u'svm_linear_2', u'svm_linear_3', u'svm_rbf_1',
       u'svm_rbf_2', u'svm_rbf_3', u'dtc_1', u'dtc_2', u'dtc_3', u'bagb_1',
       u'bagb_2', u'bagb_3', u'gradb_1', u'gradb_2', u'gradb_3', u'rdf_1',
       u'rdf_2', u'rdf_3'])
    
    for i in range(len(df)):
        #print(i)
        #print(allchampionships[i].df['country_id'].iloc[0])
        
        a=(pred_all_meth(df[i].df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction))
        b=Put_Togheter(a)
        #print(b.mean())
        tmp=tmp.append(b.mean(), ignore_index=True)
        #print(tmp)
    
    '''tmp.index=['Switzerland Super League', 'Poland Ekstraklasa',
       'Scotland Premier League', 'France Ligue 1',
       'Germany 1. Bundesliga', 'England Premier League',
       'Belgium Jupiler League', 'Portugal Liga ZON Sagres',
       'Netherlands Eredivisie', 'Italy Serie A', 'Spain LIGA BBVA']
    '''
    return tmp
        
        
    

In [1286]:
G=FINAL(allchampionships,season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

TypeError: %d format: a number is required, not str

In [2030]:
G.mean()

svm_linear_1    0.515862
svm_linear_2    0.514887
svm_linear_3    0.520850
svm_rbf_1       0.514341
svm_rbf_2       0.510355
svm_rbf_3       0.520230
dtc_1           0.498377
dtc_2           0.497134
dtc_3           0.502849
bagb_1          0.518391
bagb_2          0.492440
bagb_3          0.508048
gradb_1         0.502022
gradb_2         0.516965
gradb_3         0.506086
rdf_1           0.501985
rdf_2           0.510169
rdf_3           0.516365
dtype: float64

## Get The best Methods and Scores for Each League

In [ ]:
for i in range(len(G)):
    
    for i in range(G.iloc[i])
    print ('Best method are', b.mean()[b.mean()==max(b.mean())].index[i], 'with score', b.mean()[b.mean()==max(b.mean())][i])

    print ('Best Methods for' )
    

## SVM_LINEAR

In [495]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [496]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [497]:
c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [498]:
a['svm_linear'].mean()

0.5162414965986394

In [499]:
b['svm_linear'].mean()

0.5733843537414965

In [500]:
c['svm_linear'].mean()

0.5402483852432872

## SVM_RBF


In [501]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [502]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [503]:
a['svm_rbf'].mean()

0.601955782312925

In [504]:
b['svm_rbf'].mean()

0.601955782312925

## CART

In [505]:
dtc = DecisionTreeClassifier()
d=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

e=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

t=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')


In [507]:
d['dtc'].mean()

0.43792517006802717

In [508]:
e['dtc'].mean()

0.5260204081632655

In [509]:
#a.join(e, how='outer')

# Adaboost Classifier 

In [510]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')
c=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

In [512]:
a['bagb'].mean()

0.5034863945578232

In [513]:
b['bagb'].mean()

0.5394557823129251

# Gradient Boosting Classifier


In [514]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')
c=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

In [516]:
a['gradb'].mean()

0.4739795918367347

In [517]:
b['gradb'].mean()

0.536734693877551

# Random Forest

In [518]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf')

In [519]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf')

In [520]:
a['rdf'].mean()

0.5062925170068027

In [521]:
b['rdf'].mean()

0.486139455782313

[<__main__.MyDf instance at 0x10fdcbdd0>,
 <__main__.MyDf instance at 0x10fe925a8>,
 <__main__.MyDf instance at 0x10fea92d8>,
 <__main__.MyDf instance at 0x10d78a488>,
 <__main__.MyDf instance at 0x10df0fa28>,
 <__main__.MyDf instance at 0x10d6077e8>,
 <__main__.MyDf instance at 0x10eb7a368>,
 <__main__.MyDf instance at 0x10fe9ad88>,
 <__main__.MyDf instance at 0x10e284b90>,
 <__main__.MyDf instance at 0x10dd345a8>,
 <__main__.MyDf instance at 0x10fe9ab00>,
 <__main__.MyDf instance at 0x10dd34290>,
 <__main__.MyDf instance at 0x10feb6710>,
 <__main__.MyDf instance at 0x10d60c998>,
 <__main__.MyDf instance at 0x10d5f79e0>,
 <__main__.MyDf instance at 0x10e295fc8>,
 <__main__.MyDf instance at 0x102a135a8>,
 <__main__.MyDf instance at 0x10e345680>]

# Working on sureness

In [826]:
df=allchampionships[0].df
df=df[df['season']==season_to_be_predicted]


In [827]:
for i in range(7,30):
    X_train=df[df.H_ROUND<i].iloc[:,column_to_use_as_prediction]
    Y_train=df[df.H_ROUND<i][column_to_predict]
    X_test=df[df.H_ROUND==i].iloc[:,column_to_use_as_prediction]
    Y_test=df[df.H_ROUND==i][column_to_predict]
    svm_linear = svm.SVC(kernel='linear')
    svm_linear.fit(X_train,Y_train)
    print(svm_linear.decision_function(X_test))
    #print(svm_linear.predict(X_test))
    #print(svm_linear.score(X_test,Y_test))
    #print(Y_test)
    

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:552: ChangedBehaviorWarning: The decision_function_shape default value will change from 'ovo' to 'ovr' in 0.19. This will change the shape of the decision function returned by SVC.
  "SVC.", ChangedBehaviorWarning)


[[  1.4         -0.47385034  -2.6       ]
 [  1.          -0.15799586  -1.8       ]
 [  4.2          0.78956758  -4.6       ]
 [ 13.4          3.94811239 -12.2       ]
 [  1.          -0.15799586  -1.8       ]]
[[  3.4          1.0000357   -3.        ]
 [ 13.4         -0.99942882 -12.2       ]
 [  1.           1.57137247  -0.6       ]
 [  4.2         -0.14317333  -5.8       ]
 [ -1.4          0.99939312  -0.6       ]]
[[-0.33309724  1.22253381  2.        ]
 [ 0.99988196  0.55547167 -1.        ]
 [ 3.22186809  0.11097929 -4.33333333]
 [-3.22210418  0.99964048  4.33333333]
 [-0.99976391  1.22242595  2.66666667]]
[[ 7.4        -1.15875238 -0.9998161 ]
 [ 5.4        -1.31766032 -1.00001314]
 [ 5.2        -0.90487302 -0.99985551]
 [-2.4         0.4917746  -0.9997767 ]
 [ 7.4        -1.15875238 -0.9998161 ]]
[[ 1.         -0.59100457 -1.        ]
 [ 1.         -0.95391731 -1.        ]
 [ 1.         -1.31882001 -1.        ]
 [ 1.          0.22654408 -1.        ]
 [ 1.         -0.63686615 -1. 

In [828]:
column_to_predict='FTR'

In [829]:
X_train=df[df.H_ROUND<30].iloc[:,column_to_use_as_prediction]
Y_train=df[df.H_ROUND<30][column_to_predict]
X_test=df[df.H_ROUND==31].iloc[:,column_to_use_as_prediction]
Y_test=df[df.H_ROUND==31][column_to_predict]
svm_linear = svm.SVC(kernel='linear', probability=True)
svm_linear.fit(X_train,Y_train)
print(svm_linear.predict_proba(X_test))
print(svm_linear.predict(X_test))
print(svm_linear.score(X_test,Y_test))
print(Y_test)


[[ 0.23479978  0.28554626  0.47965396]
 [ 0.30194745  0.24401359  0.45403897]
 [ 0.29036263  0.27196488  0.43767249]
 [ 0.33082434  0.24203908  0.42713659]
 [ 0.29960692  0.25093446  0.44945862]]
['H' 'H' 'H' 'H' 'H']
0.4
1392    A
1393    H
1394    D
1395    A
1396    H
Name: FTR, dtype: object


In [825]:
svm_linear.decision_function(X_test)

NotFittedError: This AdaBoostClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [824]:
svm_linear.predict(X_test)

NotFittedError: This AdaBoostClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [351]:
svm_linear.score(X_test,Y_test)

0.80000000000000004

In [316]:
Y_test

1387    1
1388    0
1389    0
1390    0
1391    0
Name: Goal/No_Goal, dtype: int64

In [345]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
rdf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [346]:
rdf.feature_importances_

array([ 0.08142444,  0.09222371,  0.19321762,  0.17049799,  0.17541312,
        0.14064783,  0.14657529])

In [347]:
rdf.predict(X_test)

array([0, 1, 0, 1, 1])